I found a pre-trained model for a lane boundary following self-driving bot [here](https://rope.donkeycar.com/nets/1/). The model is also stored in ./all_lined_tracks_linear.h5. The idea is to finetune it with a few dense layers and our own dataset in github.com/yati-sagade/aveta-data.

In [1]:
import os
import random

from keras.models import load_model, Model
from keras.layers import Dense, Conv2D, Flatten, Dropout, Input
from matplotlib import pyplot as plt
import numpy as np
from scipy.misc import imread, imresize
from keras.callbacks import ModelCheckpoint, EarlyStopping

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [2]:
pretrained_model = load_model('all_lined_tracks_linear.h5')
pretrained_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
img_in (InputLayer)             (None, 120, 160, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 58, 78, 24)   1824        img_in[0][0]                     
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 27, 37, 32)   19232       conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 12, 17, 64)   51264       conv2d_2[0][0]                   
__________________________________________________________________________________________________
conv2d_4 (

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/models.py:318: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


I want to pop out the angle_out and throttle_out tensors and stack two dense layers, of sizes 32, 16, followed by the output layer with two units. The two hidden units can have the sigmoid activation while it is convenient for the output layer units to have tanh activation because of the nice `[-1, 1]` range, which I can map to wheel speeds in either direction quite easily. Note that an output of 1 shall correspond to a speed of 255 and -1 to a speed of -255, which is what the motot HAT library we are using for Aveta expects.


In [3]:
def build_network():
    img_in = Input(shape=(120, 160, 3), name='img_in')

    x = Conv2D(24, (5, 5), name='conv2d_1', activation='relu', strides=(2, 2))(img_in)
    x = Conv2D(32, (5, 5), name='conv2d_2', activation='relu', strides=(2, 2))(x)
    x = Conv2D(64, (5, 5), name='conv2d_3', activation='relu', strides=(2, 2))(x)
    x = Conv2D(64, (3, 3), name='conv2d_4', activation='relu', strides=(2, 2))(x)
    x = Conv2D(64, (3, 3), name='conv2d_5', activation='relu', strides=(1, 1))(x)
    x = Flatten(name='flattened')(x)
    x = Dense(100, activation='relu', name='dense_1')(x)
    x = Dropout(rate=0.1, name='dropout_1')(x)
    x = Dense(50, activation='relu', name='dense_2')(x)
    x = Dropout(rate=0.1, name='dropout_2')(x)
    # This is the beginning of our additions
    x = Dense(32, activation='sigmoid', name='dense_3')(x)
    x = Dropout(rate=0.1, name='dropout_3')(x)
    x = Dense(16, activation='sigmoid', name='dense_4')(x)
    x = Dropout(rate=0.1, name='dropout_4')(x)

    speeds_out = Dense(2, activation='tanh', name='dense_5')(x)

    model = Model(inputs=[img_in], outputs=[speeds_out])
    return model

In [4]:
model = build_network()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
img_in (InputLayer)          (None, 120, 160, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 58, 78, 24)        1824      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 27, 37, 32)        19232     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 12, 17, 64)        51264     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 5, 8, 64)          36928     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 3, 6, 64)          36928     
_________________________________________________________________
flattened (Flatten)          (None, 1152)              0         
__________

I do not know of a way to generate python code from a stored Keras model, so I had to dig into the JSON representation of the model and write out the model by hand. Of course I've left out the last two layers of the original model, and instead (after the dropout_2 layer) added my own dense layers. I don't have a huge dataset, so I've included dropout layers to guard against overfitting. This may not be needed.

Now to transfer the weights of the pretrained model onto our model's first layers:

In [5]:
for layer in model.layers:
    if layer.name == 'dense_3':
        break # Starting here, it's our layers
    weights = pretrained_model.get_layer(name=layer.name).get_weights()
    layer.set_weights(weights)
    layer.trainable = False
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
img_in (InputLayer)          (None, 120, 160, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 58, 78, 24)        1824      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 27, 37, 32)        19232     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 12, 17, 64)        51264     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 5, 8, 64)          36928     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 3, 6, 64)          36928     
_________________________________________________________________
flattened (Flatten)          (None, 1152)              0         
__________

Now to massage the training data so that we have the following:
- A suitable train/test split
- Speeds are in speeds.txt, and need to be normalized in the `[-1, 1]` by simply dividing by 255.

In [7]:
import os

TEST_MODE = False
AVETA_DATA_DIR = os.path.expanduser('~/aveta-data')

imdir = os.path.join(AVETA_DATA_DIR, 'generated')
imfiles = [f for f in os.listdir(imdir) if f.endswith('.jpg')]
random.shuffle(imfiles)

if TEST_MODE:
    imfiles = imfiles[:100]

speedfile = os.path.join(imdir, 'speeds.txt')
speeds = {}
with open(speedfile) as fp:
    for line in fp:
        imfile, _, _, left_speed, right_speed = line.strip().split(',')
        speeds[imfile] = [left_speed, right_speed]
    
nbtest = int(len(imfiles)*0.1)
test_filenames, train_filenames = imfiles[:nbtest], imfiles[nbtest:]
test_x = np.array([
    imresize(imread(os.path.join(imdir, f)), (120, 160))
    for f in test_filenames
])
train_x = np.array([
    imresize(imread(os.path.join(imdir, f)), (120, 160))
    for f in train_filenames
])
test_y, train_y = [np.array([speeds[f] for f in fileset], dtype=np.float) / 255.0
                    for fileset in (test_filenames, train_filenames)]

print('Shapes: train_x: {}, train_y: {}, test_x: {}, test_y: {}'.format(
    train_x.shape, train_y.shape, test_x.shape, test_y.shape
));

print(train_y.min(), train_y.max())



/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:24: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:24: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:28: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:28: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


Shapes: train_x: (7065, 120, 160, 3), train_y: (7065, 2), test_x: (785, 120, 160, 3), test_y: (785, 2)
-0.09411764705882353 0.6588235294117647


At this point, the model can be trained

In [10]:
import datetime
now = datetime.datetime.now()
checkpoint_filename = 'aveta-checkpoint-{}'.format(now.strftime('%Y%m%d%H%M%S'))
save_best = ModelCheckpoint(checkpoint_filename,
                            save_best_only=True,
                            verbose=1,
                            mode='min')

early_stop = EarlyStopping(monitor='val_loss',
                           min_delta=0,
                           patience=5,
                           verbose=0,
                           mode='auto')

cbs = [save_best, early_stop]

batch_size = 320 if not TEST_MODE else 64
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(train_x,
          train_y,
          nb_epoch=100,
          validation_split=0.1,
          callbacks=cbs,
          batch_size=batch_size)

test_eval = model.evaluate(test_x, test_y, batch_size=batch_size)
print(test_eval)

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:24: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 6358 samples, validate on 707 samples
Epoch 1/100
6358/6358 [==============================] - 3s 492us/step - loss: 0.0064 - val_loss: 0.0059

Epoch 00001: val_loss improved from inf to 0.00588, saving model to aveta-checkpoint-20180510200434
Epoch 2/100
6358/6358 [==============================] - 3s 419us/step - loss: 0.0063 - val_loss: 0.0059

Epoch 00002: val_loss did not improve from 0.00588
Epoch 3/100
6358/6358 [==============================] - 3s 416us/step - loss: 0.0062 - val_loss: 0.0059

Epoch 00003: val_loss did not improve from 0.00588
Epoch 4/100
6358/6358 [==============================] - 3s 416us/step - loss: 0.0062 - val_loss: 0.0059

Epoch 00004: val_loss improved from 0.00588 to 0.00587, saving model to aveta-checkpoint-20180510200434
Epoch 5/100
6358/6358 [==============================] - 3s 414us/step - loss: 0.0062 - val_loss: 0.0059

Epoch 00005: val_loss did not improve from 0.00587
Epoch 6/100
6358/6358 [==============================] - 3s 412us/